In [7]:
import sys
sys.path.append('../')
from Utils import *
from einops import rearrange
import matplotlib.pyplot as plt

In [8]:
# Read one subject/case from a directory containing the corresponding .mat files
data_dir = '/mnt/nas/nas3/openData/rawdata/4dFlow/ChallengeData/DemoData'

# `load_mat` takes the .mat file path and variable name (`key`) as input.
# It returns a lazy HDF5-like handle (h5py-style): data is not loaded into memory
# until you index it.
# load a pre-reconstructed image (assumed to be already reconstructed)
kdata_full = load_mat(data_dir + '/kdata_full.mat', key='kdata_full')
coilmap    = load_mat(data_dir + '/coilmap.mat',    key='coilmap')
segmask    = load_mat(data_dir + '/segmask.mat',    key='segmask')
params     = read_params_csv(data_dir + '/params.csv')
img_csllr = load_coo_npz(f"{data_dir}/img_ktGaussian10.npz", as_dense=True)
# Use [()] to materialize the entire dataset into RAM as a NumPy array.
kdata_full = kdata_full[()]
coilmap    = coilmap[()]
segmask    = segmask[()]
img_csllr = img_csllr[()]

img = np.sum(k2i_numpy(kdata_full, ax=[-1, -2, -3]) * np.conj(coilmap), axis=-4)
Nv, Nt, Nc, SPE, PE, FE = kdata_full.shape

VENC = params['VENC']

In [9]:
corr_maps = execute_MSAC(img, corr_fit_order=3, th=0.1)
img_corr = img.copy()
img_corr[1:] *= np.exp(1j * -corr_maps)
mag_gt, flow_gt = complex2magflow(img_corr, venc=np.array(VENC))

In [10]:
img_csllr_corr = img_csllr.copy()
img_csllr_corr[1:] *= np.exp(1j * -corr_maps)
mag_csllr, flow_csllr = complex2magflow(img_csllr_corr, venc=np.array(VENC))

In [11]:
# --- Quality metrics (CS-LLR reconstruction vs. Ground Truth) ---
# SSIM   in [0, 1]   (higher is better): structural similarity, averaged within segmask (segmask==1).
# nRMSE  in [0, 1]   (lower is better): RMSE normalized by the GT peak value, computed within segmask.
# RelErr in [0, 1]   (lower is better): relative error of vector-field magnitudes, computed within segmask.
# AngErr in degrees (°) (lower is better): mean angular (direction) error of vector fields, computed within segmask.
ssim_val   = SSIM(mag_csllr, mag_gt, segmask)
nrmse_val  = nRMSE(mag_csllr, mag_gt, segmask)
relerr_val = RelErr(flow_csllr, flow_gt, segmask)
angerr_val = AngErr(flow_csllr, flow_gt, segmask)

print(f"SSIM   (mag, 0–1): {ssim_val:.6f}")
print(f"nRMSE  (mag, 0–1): {nrmse_val:.6f}")
print(f"RelErr (flow, 0–1): {relerr_val:.6f}")
print(f"AngErr (flow, °): {angerr_val:.3f}°")

SSIM   (mag, 0–1): 0.989542
nRMSE  (mag, 0–1): 0.042582
RelErr (flow, 0–1): 0.263255
AngErr (flow, °): 37.754°
